In [50]:
quit()

: 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
yarn = SparkSession\
        .builder\
        .master('yarn')\
        .appName('twitter_yarn')\
        .config('spark.executor.instances', 4)\
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
        .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bigdata/.ivy2/cache
The jars for the packages stored in: /home/bigdata/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dc51f40b-5e8d-4856-b74a-ca531fffb539;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 352ms :: artifacts dl 31ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifac

23/03/23 11:17:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/23 11:17:50 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/03/23 11:18:07 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb.spark_mongo-spark-connector_2.12-3.0.1.jar added multiple times to distributed cache.
23/03/23 11:18:07 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb_mongodb-driver-sync-4.0.5.jar added multiple times to distributed cache.
23/03/23 11:18:07 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb_bson-4.0.5.jar added multiple times to distributed cache.
23/03/23 11:18:07 WARN Client: Same path resource file:///home/bigdata/.ivy2/jars/org.mongodb_mongodb-driver-core-4.0.5.jar added multiple times to distributed cache.


In [3]:
yarn.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '4'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.ui.proxyBase', '/proxy/application_1679537503616_0001'),
 ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1'),
 ('spark.drive

In [4]:
%%time
df = yarn.read.format('mongo')\
               .option('uri', 'mongodb://192.168.56.101:27017/twitter.sample')\
               .load()

df.createOrReplaceTempView('tweets')

CPU times: user 0 ns, sys: 15.6 ms, total: 15.6 ms
Wall time: 19.5 s


In [5]:
# 전체 데이터 수 확인
query ='''
SELECT count(*)
FROM tweets
'''
yarn.sql(query).show()

+--------+
|count(1)|
+--------+
| 7739109|
+--------+



In [9]:
%%time
# 데이터 확인

query = """
  SELECT *
  FROM tweets
"""
yarn.sql(query).show(5)

+--------------------+--------------------+--------------------+
|                 _id|          _timestamp|                data|
+--------------------+--------------------+--------------------+
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
|{64102551db4069ef...|2023-03-14T16:42:...|{2023-03-14T07:42...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

CPU times: user 2.22 ms, sys: 7.74 ms, total: 9.97 ms
Wall time: 13.7 s


In [15]:
%%time
#언어별 count 확인

query = '''
  SELECT data.lang as lang, count(*) count
  FROM tweets
  WHERE data.lang is not NULL
  GROUP BY lang
  ORDER BY count DESC;
'''

yarn.sql(query).show(5)

+----+-------+
|lang|  count|
+----+-------+
|  en|2197064|
|  ja|1708222|
|  th| 484936|
|  es| 467246|
|  ko| 316200|
+----+-------+
only showing top 5 rows

CPU times: user 49.9 ms, sys: 61 ms, total: 111 ms
Wall time: 1min 5s


In [16]:
%%time
#영어로 작성된 tweet만 추출

query = """
  SELECT
    DATE_FORMAT(_timestamp, "yyyy-MM-dd HH:mm:ss") as time,
    data.text as text
  FROM tweets
  WHERE data.lang = 'en';
"""

en_tweets = yarn.sql(query)
en_tweets.show(5)

+-------------------+--------------------+
|               time|                text|
+-------------------+--------------------+
|2023-03-14 16:42:09|@erdincciftci Goo...|
|2023-03-14 16:42:09|@CheapremeBrand A...|
|2023-03-14 16:42:09|RT @Joon_record: ...|
|2023-03-14 16:42:09|RT @TheRavenCrypt...|
|2023-03-14 16:42:09|@gareth_lane He i...|
+-------------------+--------------------+
only showing top 5 rows

CPU times: user 4.39 ms, sys: 7.34 ms, total: 11.7 ms
Wall time: 7.82 s


In [17]:
%%time
#tweet의 text를 단어별로 split하는 함수
def text_split(row):
  for word in row.text.split():
    yield Row(time=row.time, word=word)
  

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 6.2 µs


In [18]:
%%time
#샘플하나 추출
en_tweets.rdd.take(1)

CPU times: user 2.31 ms, sys: 49.4 ms, total: 51.7 ms
Wall time: 3.74 s


[Row(time='2023-03-14 16:42:09', text='@erdincciftci Good morning forever Erdin 🙏🔥')]

In [19]:
%%time
#단어별로 split한뒤에 5개 show
en_tweets.rdd.flatMap(text_split).take(5)

CPU times: user 667 µs, sys: 13.2 ms, total: 13.8 ms
Wall time: 628 ms


[Row(time='2023-03-14 16:42:09', word='@erdincciftci'),
 Row(time='2023-03-14 16:42:09', word='Good'),
 Row(time='2023-03-14 16:42:09', word='morning'),
 Row(time='2023-03-14 16:42:09', word='forever'),
 Row(time='2023-03-14 16:42:09', word='Erdin')]

In [20]:
%%time
#단어별로 split한뒤에 데이터프레임으로 변경후 5개 show
en_tweets.rdd.flatMap(text_split).toDF().show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-14 16:42:09|@erdincciftci|
|2023-03-14 16:42:09|         Good|
|2023-03-14 16:42:09|      morning|
|2023-03-14 16:42:09|      forever|
|2023-03-14 16:42:09|        Erdin|
+-------------------+-------------+
only showing top 5 rows

CPU times: user 107 ms, sys: 498 ms, total: 605 ms
Wall time: 6.52 s


In [21]:
%%time
#단어별로 분리하고 데이터프레임으로 추출
words = en_tweets.rdd.flatMap(text_split).toDF()
words.createOrReplaceTempView('words')


CPU times: user 10.7 ms, sys: 4.21 ms, total: 14.9 ms
Wall time: 601 ms


In [23]:
%%time
#words의 상위 5개 show
words.show(5)

+-------------------+-------------+
|               time|         word|
+-------------------+-------------+
|2023-03-14 16:42:09|@erdincciftci|
|2023-03-14 16:42:09|         Good|
|2023-03-14 16:42:09|      morning|
|2023-03-14 16:42:09|      forever|
|2023-03-14 16:42:09|        Erdin|
+-------------------+-------------+
only showing top 5 rows

CPU times: user 5.52 ms, sys: 171 µs, total: 5.69 ms
Wall time: 1.61 s


In [27]:
%%time
#데이터프레임에서 개수 확인
words.count()

CPU times: user 30.2 ms, sys: 95.3 ms, total: 125 ms
Wall time: 1min 21s


34827421

In [24]:
%%time
#words를 hdfs warehouse에 저장
words.write.saveAsTable('yarn_twitter_sample_words')

CPU times: user 41.2 ms, sys: 92.5 ms, total: 134 ms
Wall time: 1min 47s


In [34]:
!hdfs dfs -ls spark-warehouse/yarn_twitter_sample_words | grep _SUCCESS

-rw-r--r--   3 bigdata supergroup          0 2023-03-23 11:34 spark-warehouse/yarn_twitter_sample_words/_SUCCESS


In [38]:
%%time
#hdfs warehouse에 저장된 테이블 개수파악
yarn.table('yarn_twitter_sample_words').count()

CPU times: user 0 ns, sys: 10.4 ms, total: 10.4 ms
Wall time: 3.45 s


34827421

물리적인 테이블의 형태로 보관하니 집계 시간이 빨라짐

In [36]:
%%time
query = '''
    SELECT word, count(*) count
    FROM words
    GROUP BY word
    ORDER BY count DESC
'''

yarn.sql(query).show(5)

+----+-------+
|word|  count|
+----+-------+
|  RT|1232866|
| the| 770781|
|  to| 601721|
|   a| 435703|
| and| 416036|
+----+-------+
only showing top 5 rows

CPU times: user 35 ms, sys: 101 ms, total: 136 ms
Wall time: 1min 27s


In [37]:
%%time
query = '''
    SELECT word, count(*) count
    FROM yarn_twitter_sample_words
    GROUP BY word
    ORDER BY count DESC
'''

yarn.sql(query).show(5)

+----+-------+
|word|  count|
+----+-------+
|  RT|1232866|
| the| 770781|
|  to| 601721|
|   a| 435703|
| and| 416036|
+----+-------+
only showing top 5 rows

CPU times: user 381 µs, sys: 36.7 ms, total: 37.1 ms
Wall time: 12.3 s


In [39]:
%%time
yarn.sql('SELECT * FROM yarn_twitter_sample_words').show(5)

+-------------------+--------------+
|               time|          word|
+-------------------+--------------+
|2023-03-15 22:15:30|            RT|
|2023-03-15 22:15:30|@mckaywrigley:|
|2023-03-15 22:15:30|            We|
|2023-03-15 22:15:30|           are|
|2023-03-15 22:15:30|           all|
+-------------------+--------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 4.42 ms, total: 4.42 ms
Wall time: 591 ms


In [40]:
%%time
yarn.table('yarn_twitter_sample_words').count()

CPU times: user 0 ns, sys: 6.03 ms, total: 6.03 ms
Wall time: 1.21 s


34827421

In [41]:
%%time
query = '''
 SELECT substr(time, 1, 13) time,
 word,
 count(*) count
 FROM yarn_twitter_sample_words
 GROUP BY time, word
''' 
yarn.sql(query).count()

CPU times: user 12.9 ms, sys: 23.2 ms, total: 36.1 ms
Wall time: 32.8 s


28603658

In [43]:
%%time
#등록 횟수가 적은 단어의 수 조사

query = '''
    SELECT t.count, count(*) words
    FROM (
    SELECT word, count(*) count FROM yarn_twitter_sample_words
    GROUP BY word
    ) t
    GROUP BY 1
    ORDER BY 1
'''

yarn.sql(query).show(5)

+-----+-------+
|count|  words|
+-----+-------+
|    1|2259387|
|    2| 321173|
|    3| 134057|
|    4|  75277|
|    5|  48716|
+-----+-------+
only showing top 5 rows

CPU times: user 7.31 ms, sys: 10.9 ms, total: 18.2 ms
Wall time: 8.28 s


In [44]:
%%time
# 단어를 카테고리로 나누는 디멘전 테이블

query = '''
 SELECT word, count,
     IF(count > 1000, word, concat('COUNT=', count)) category
 FROM (
     SELECT word, count(*) count FROM yarn_twitter_sample_words
     GROUP BY 1
 ) t
'''

yarn.sql(query).show(10)

+--------------------+-----+--------+
|                word|count|category|
+--------------------+-----+--------+
|            time(any|    1| COUNT=1|
|https://t.co/0Bp6...|    1| COUNT=1|
|                hope|10093|    hope|
|                some|28064|    some|
|              online| 2839|  online|
|                 ...| 5368|     ...|
|            #杭州spa| 1289|#杭州spa|
|               still|24013|   still|
|     @CaitlynMFoster|    1| COUNT=1|
|           @FrogBorg|    3| COUNT=3|
+--------------------+-----+--------+
only showing top 10 rows

CPU times: user 5.21 ms, sys: 15.1 ms, total: 20.3 ms
Wall time: 8.77 s


In [45]:
%%time
yarn.sql(query).createOrReplaceTempView('word_category')

CPU times: user 3.08 ms, sys: 3.48 ms, total: 6.56 ms
Wall time: 163 ms


In [46]:
%%time
# spark 데이터프레임
# 1시간마다 카테고리별로 그룹화하여 집계
query = '''
 SELECT substr(a.time, 1, 13) time, b.category, count(*) count
 FROM words a
 LEFT JOIN word_category b ON a.word = b.word
 GROUP BY 1, 2
'''

yarn.sql(query).show(5)
yarn.sql(query).count()

+-------------+---------+-----+
|         time| category|count|
+-------------+---------+-----+
|2023-03-21 16|  COUNT=3| 8350|
|2023-03-16 12| COUNT=21| 1384|
|2023-03-17 00|COUNT=118|  111|
|2023-03-16 01|COUNT=109|  192|
|2023-03-18 14|COUNT=300|   42|
+-------------+---------+-----+
only showing top 5 rows



CPU times: user 128 ms, sys: 211 ms, total: 339 ms
Wall time: 3min 51s


220243

In [47]:
%%time
# hdfs에 저장되어있는 테이블
# 1시간마다 카테고리별로 그룹화하여 집계
query = '''
 SELECT substr(a.time, 1, 13) time, b.category, count(*) count
 FROM yarn_twitter_sample_words a
 LEFT JOIN word_category b ON a.word = b.word
 GROUP BY 1, 2
'''

yarn.sql(query).show(5)
yarn.sql(query).count()

+-------------+---------+-----+
|         time| category|count|
+-------------+---------+-----+
|2023-03-16 17|COUNT=121|  321|
|2023-03-21 16|  COUNT=3| 8350|
|2023-03-16 00| COUNT=29| 1609|
|2023-03-21 19|COUNT=858|   85|
|2023-03-17 13|COUNT=117|  273|
+-------------+---------+-----+
only showing top 5 rows



CPU times: user 57 ms, sys: 40.4 ms, total: 97.4 ms
Wall time: 57.8 s


220243

In [48]:
%%time
result = yarn.sql(query).toPandas()
result.head()

CPU times: user 265 ms, sys: 1.84 s, total: 2.11 s
Wall time: 41.2 s


,time,category,count
0,2023-03-21 16,COUNT=3,8350
1,2023-03-16 12,COUNT=21,1384
2,2023-03-17 00,COUNT=118,111
3,2023-03-16 01,COUNT=109,192
4,2023-03-18 14,COUNT=300,42


In [49]:
result.tail()

,time,category,count
220238,2023-03-21 00,dare,3
220239,2023-03-16 22,fool,30
220240,2023-03-15 22,haven't,58
220241,2023-03-22 00,picture,6
220242,2023-03-17 12,piece,50
